### 1. Objective of This Notebook 

This notebook answers the question:

“Based on Apple’s historical financial performance, what could the future look like under different scenarios?”

We will:

* Forecast key financial metrics

* Build realistic scenarios

* Assess upside and downside risks

* Support strategic and investment decisions

### 2. Forecasting Scope (What We Will Predict)

We focus on high-impact, forecastable metrics:

Core Forecast Variables

* Revenue

* Net Income

* Operating Cash Flow

* Free Cash Flow

Why these?

* They drive valuation

* They reflect operational strength

* They connect directly to investor decisions

### 3. Data Preparation for Forecasting

In [ ]:
# Load Required Libraries
import pandas as pd
import numpy as np


In [5]:
# Load Financial Statements
income = pd.read_csv("apple_income_statement.csv", index_col=0, parse_dates=True)
balance = pd.read_csv("apple_balance_sheet.csv", index_col=0, parse_dates=True)
cashflow = pd.read_csv("apple_cash_flow.csv", index_col=0, parse_dates=True)


In [21]:
# Sort Data by Year
income = income.sort_index()
balance = balance.sort_index()
cashflow = cashflow.sort_index()
years = income.index.year

In [ ]:
# sanity check
print(income.shape)
print(balance.shape)
print(cashflow.shape)


(5, 40)
(5, 70)
(5, 54)


#### 4. Baseline Growth Rate Estimation

We use historical CAGR as the baseline assumption.

In [ ]:
# Revenue Compound Annual Growth Rate (CAGR)
income = income.sort_index()
cashflow = cashflow.sort_index()

# Slice data for 2022-2025
income_22_25 = income.loc['2022-01-01':'2025-12-31']
cashflow_22_25 = cashflow.loc['2022-01-01':'2025-12-31']

# CAGR function
def calculate_cagr(series):
    start, end = series.iloc[0], series.iloc[-1]
    periods = len(series) - 1
    return (end / start) ** (1 / periods) - 1

# Compute CAGR for 2022-2025
revenue_cagr = calculate_cagr(income_22_25["Total Revenue"])
net_income_cagr = calculate_cagr(income_22_25["Net Income"])
fcf_cagr = calculate_cagr(cashflow_22_25["Free Cash Flow"])

# Print results
print(f"Revenue CAGR (2022-2025): {revenue_cagr:.2%}")
print(f"Net Income CAGR (2022-2025): {net_income_cagr:.2%}")
print(f"Free Cash Flow CAGR (2022-2025): {fcf_cagr:.2%}")


Revenue CAGR (2022-2025): 1.81%
Net Income CAGR (2022-2025): 3.92%
Free Cash Flow CAGR (2022-2025): -3.95%


Insights:

Revenue CAGR (1.81%): Revenue is growing slowly, indicating modest top-line expansion.

Net Income CAGR (3.92%): Profits are growing faster than revenue, showing improved efficiency and cost control.

FCF CAGR (-3.95%): Free cash flow is declining, suggesting higher capital spending or cash outflows despite rising profits.

Overall: Profitability is improving, but cash generation is under pressure—something to watch closely.

In [ ]:
# Forecast Years
forecast_years = 5
last_year = years[-1]
future_years = [last_year + i for i in range(1, forecast_years + 1)]

print(future_years)


[np.int32(2026), np.int32(2027), np.int32(2028), np.int32(2029), np.int32(2030)]


In [31]:
# Base Forecast
base_forecast = pd.DataFrame(index=future_years)

base_forecast["Revenue"] = [
    income["Total Revenue"].iloc[-1] * (1 + revenue_cagr) ** i
    for i in range(1, forecast_years + 1)
]

base_forecast["Net Income"] = [
    income["Net Income"].iloc[-1] * (1 + net_income_cagr) ** i
    for i in range(1, forecast_years + 1)
]

base_forecast["Free Cash Flow"] = [
    cashflow["Free Cash Flow"].iloc[-1] * (1 + fcf_cagr) ** i
    for i in range(1, forecast_years + 1)
]

base_forecast


,Revenue,Net Income,Free Cash Flow
2026,4.237041e+11,1.164022e+11,9.487058e+10
2027,4.313839e+11,1.209666e+11,9.112787e+10
2028,4.392028e+11,1.257100e+11,8.753282e+10
2029,4.471635e+11,1.306395e+11,8.407960e+10
2030,4.552685e+11,1.357622e+11,8.076260e+10


Here’s a brief analysis of the 2026–2030 projections:

* Revenue: Gradually increasing from $423.7B to $455.3B, 1.8–2% annual growth. The growth is steady but moderate, suggesting a mature business.

* Net Income: Rises faster than revenue from $116.4B to $135.8B, indicating improving profitability and operational efficiency.

* Free Cash Flow (FCF): Declining from $94.9B to $80.8B, despite higher profits, suggesting increasing capital expenditures or other cash uses.

Overall: The company shows stable revenue growth and strong profit expansion, but cash flow generation is under slight pressure. This pattern is typical for a mature company investing in growth or assets.

In [35]:
scenarios = {
    "Bear": 0.7,
    "Base": 1.0,
    "Bull": 1.3
}

scenario_forecast = {}

for scenario, factor in scenarios.items():
    scenario_forecast[scenario] = [
        income["Total Revenue"].iloc[-1] * (1 + revenue_cagr * factor) ** i
        for i in range(1, forecast_years + 1)
    ]

scenario_revenue = pd.DataFrame(
    scenario_forecast, index=future_years
)

scenario_revenue


,Bear,Base,Bull
2026,4.214411e+11,4.237041e+11,4.259670e+11
2027,4.267883e+11,4.313839e+11,4.360040e+11
2028,4.322033e+11,4.392028e+11,4.462776e+11
2029,4.376870e+11,4.471635e+11,4.567932e+11
2030,4.432402e+11,4.552685e+11,4.675566e+11


Here’s a quick analysis of the projected revenue scenarios for 2026–2030:

* Bear case: Slowest growth, reaching $443.2B in 2030. Indicates conservative expectations or potential headwinds.

* Base case: Moderate growth, reaching $455.3B in 2030. Represents the most likely scenario based on current trends.

* Bull case: Strongest growth, reaching $467.6B in 2030. Suggests optimistic outcomes from higher demand, pricing power, or operational improvements.

Overall: All scenarios show steady revenue growth, with Base and Bull cases reflecting moderate to strong expansion, while the Bear case signals slower but still positive growth. This gives a good range for planning and risk assessment.

In [39]:
scenario_fcf = {}

for scenario, factor in scenarios.items():
    scenario_fcf[scenario] = [
        cashflow["Free Cash Flow"].iloc[-1] * (1 + fcf_cagr * factor) ** i
        for i in range(1, forecast_years + 1)
    ]

scenario_fcf_df = pd.DataFrame(
    scenario_fcf, index=future_years
)

scenario_fcf_df


,Bear,Base,Bull
2026,9.603951e+10,9.487058e+10,9.370165e+10
2027,9.338733e+10,9.112787e+10,8.889609e+10
2028,9.080840e+10,8.753282e+10,8.433698e+10
2029,8.830068e+10,8.407960e+10,8.001169e+10
2030,8.586222e+10,8.076260e+10,7.590822e+10


Here’s a brief analysis of the projected Free Cash Flow (FCF) scenarios for 2026–2030:

* Bear case: Highest FCF, starting $96.0B in 2026 and declining to $85.9B in 2030. Suggests conservative investments or lower spending, preserving cash.

* Base case: Moderate decline from $94.9B → $80.8B, reflecting typical capital expenditures and cash outflows for a growing company.

* Bull case: Lowest FCF, dropping from $93.7B → $75.9B, likely due to aggressive investment or expansion plans.

Overall: Free cash flow is projected to decline across all scenarios, even though revenue may grow. This indicates rising capital requirements or strategic investments despite profitability, which is common in mature companies pursuing growth.